In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from dotenv import load_dotenv
import os
import base64
from requests import post
import json
import tekore as tk

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
cid = '279b5795a43445008a27c72111ad8e51'
secret = '244ed65b159d47b9be9482d50ada761d'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
def get_token():
    auth = cid + ":" + secret
    bytes = auth.encode("utf-8")
    auth_base64 = str(base64.b64encode(bytes), "utf-8")

    url = 'https://accounts.spotify.com/api/token'
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

In [4]:
def call_playlist(creator, playlist_id):
    token = get_token()
    spotify = tk.Spotify(token)

    # Tekore segment
    playlist_data = []
    num = 1
    first_items = spotify.playlist_items(playlist_id)
    for item in spotify.all_items(first_items):
        track_id = item.track.id
        artist = item.track.artists[0].name
        song_name = item.track.name
        track_info = {
            num: {
                "track_id": track_id,
                "artist": artist,
                "song_name": song_name
            }
        }
        num += 1
        playlist_data.append(track_info)

    # Spotipy segment
    playlist_features_list = ["artist", "album", "track_name", "track_id", "danceability", "energy", "loudness", "mode", "speechiness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]

    offset = 0
    playlist_items = sp.user_playlist_tracks(creator, playlist_id, offset=offset)["items"]

    while playlist_items:
        for index, track in enumerate(playlist_items, start=1):
            playlist_features = {}
            playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
            playlist_features["album"] = track["track"]["album"]["name"]
            playlist_features["track_name"] = track["track"]["name"]
            playlist_features["track_id"] = track["track"]["id"]

            # Get audio features
            audio_features = sp.audio_features(playlist_features["track_id"])[0]
            for feature in playlist_features_list[4:]:
                playlist_features[feature] = audio_features[feature]

            playlist_data.append({index: playlist_features})

        # Increment offset to fetch the next set of tracks
        offset += len(playlist_items)
        playlist_items = sp.user_playlist_tracks(creator, playlist_id, offset=offset)["items"]

    return playlist_data

In [5]:
call_playlist("spotify","4n8ardwLpQP9eBZzQIETWi")

[{1: {'track_id': '2l3dFxtoSMgMQVyRIUHjpz',
   'artist': 'Mitraz',
   'song_name': 'Akhiyaan'}},
 {2: {'track_id': '1yn5VIHdxIlVMPRdQzR2sm',
   'artist': 'Evan Call',
   'song_name': 'Across the Violet Sky'}},
 {3: {'track_id': '3G68NLecRlUYbO8OAs4YKb',
   'artist': 'Mitraz',
   'song_name': 'Kabhi Na Kabhi'}},
 {4: {'track_id': '7hADNKBDHS8MblpUzLebLf',
   'artist': 'Anirudh Ravichander',
   'song_name': 'Hoyna Hoyna (From "Gang Leader")'}},
 {5: {'track_id': '7k1HoUdskuBhyWvm7hPctM',
   'artist': 'Hiroyuki Sawano',
   'song_name': 'Call of Silence'}},
 {6: {'track_id': '1VDDoa82Oktn7FDPKBcsw8',
   'artist': 'Adnan Sami',
   'song_name': 'O Madhu'}},
 {7: {'track_id': '2cDyQDAoP0cPQXzaviXu2f',
   'artist': 'Bharatt-Saurabh',
   'song_name': 'Vellake'}},
 {8: {'track_id': '2LxyYFrPtuCQvXGET5DRqM',
   'artist': 'Susumu Hirasawa',
   'song_name': 'Gats'}},
 {9: {'track_id': '7kJVLzF7COXgFVlKRaYwoK',
   'artist': 'Maran',
   'song_name': 'Shades of Kadhal'}},
 {10: {'track_id': '0Z3zGNsyg